# Advanced Machine Learning @ UDD
### Instructor: Visiting Professor Rossano Schifanella

### References:
* **scikit-learn tutorial** at [SciPy 2017](https://github.com/amueller/scipy-2017-sklearn) by [Alexandre Gramfort](http://http://alexandre.gramfort.net/)  [@agramfort](https://twitter.com/agramfort) (Inria, Université Paris-Saclay) and [Andreas Mueller](http://amuller.github.io) [@amuellerml](https://twitter.com/amuellerml) (Columbia University). See the book [Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do) for more details. 
* **Data Mining, Statistical Modeling and Machine Learning** class by Dr. Ciro Cattuto, Dr. Laetitia Gauvin, Dr. André Panisson (ISI Foundation, Turi, Italy)

#### Install requires python libraries

We use pip but the same works for "conda".

>pip install scipy

>pip install pandas

>pip install scikit-learn

optional: 

>pip install seaborn

Mac users: please try installing graphviz using homebrew instead of anaconda

We are going to need a lot of Python packages, so let's start by importing all of them.

In [ ]:
# Import the libraries we will be using
import os
import numpy as np
import matplotlib.pylab as plt
from IPython.display import Image
import graphviz 

import sklearn

%matplotlib inline


We are also going to do a lot of repetitive stuff, so let's predefine some useful functions.

In [ ]:
# A function that gives a visual representation of the decision tree
def show_decision_tree(model):
    dot_data = tree.export_graphviz(decision_tree, out_file=None) 
    graph = graphviz.Source(dot_data) 
#     To save on a PDF file
#     graph.render("iris")
    return graph

Import the iris dataset

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

X, y = iris.data, iris.target

In [ ]:
print(iris.keys())
print(iris.target_names)
print(iris.feature_names)

### Useful features
Let's take a look at one of our features -- `"petal length (cm)"`. Is this feature useful? Let's plot the possible values of `"petal_length"` and color code our target variable.

In [ ]:
feature_index = iris.feature_names.index('petal length (cm)')
colors = ['blue', 'red', 'green']

for label, color in zip(range(len(iris.target_names)), colors):
    plt.hist(iris.data[iris.target==label, feature_index], 
             label=iris.target_names[label],
             color=color)
plt.xlabel(iris.feature_names[feature_index])
plt.legend(loc='upper right')
plt.show()

Is `"petal_length"` actually useful? Let's quantify it.

**Entropy** ($H$) and **information gain** ($IG$) are crucial in determining which features are the most informative. Given the data, it is fairly straight forward to calculate both of these.

<table style="border: 0px">
<tr style="border: 0px;background:'white'"">
<td style="border: 0px;background:'white'""><img src="images/dsfb_0304.png" height=80% width=80%>
Figure 3-4. Splitting the "write-off" sample into two segments, based on splitting the Balance attribute (account balance) at 50K.</td>
<td style="border: 0px; width: 30px"></td>
<td style="border: 0px"><img src="images/dsfb_0305.png" height=75% width=75%>
Figure 3-5. A classification tree split on the three-values Residence attribute.</td>
</tr>
</table>

In [ ]:
def entropy(target):
    '''
    Formula: entropy = -p1*log(p1) - p2*log(p2) - ...
    '''
    # Get the number of users
    n = len(target)
    # Count how frequently each unique value occurs
    un,counts=np.unique(target,return_counts=True)
    # Initialize entropy
    entropy = 0
    # If the split is perfect, return 0
    if len(counts) <= 1 or 0 in counts:
        return entropy
    # Otherwise, for each possible value, update entropy
    for count in counts:
        entropy += math.log(float(count)/n, len(counts)) * count/n
    # Return entropy
    return -1 * entropy

def information_gain(feature, target, threshold):
    '''
    Formula: IG(parent,children)=entropy(parent) - [p(c1)*entropy(c1) + p(c2)*entropy(c2) + ...]
    '''
    # Dealing with numpy arrays makes this slightly easier
    target = np.array(target)
    feature = np.array(feature)
    # Cut the feature vector on the threshold
    feature = (feature < threshold)
    # Initialize information gain with the parent entropy
    ig = entropy(target)
    # For both sides of the threshold, update information gain
    for level, count in zip([0, 1], np.bincount(feature).astype(float)):
        ig -= count/len(feature) * entropy(target[feature == level])
    # Return information gain
    return ig

Now that we have a way of calculating $H$ and $IG$, let's pick a threshold, split `"petal_length"`, and calculate $IG$.

In [ ]:
print(X[:,2])
print(y)


In [ ]:
import math

threshold = 4.7
print("IG = %.4f with a thresholding of %.2f" %(information_gain(X[:, 2], y,threshold), threshold))

To be more precise, we can iterate through all values and find the best split.

In [ ]:
def best_threshold(X, y):
    maximum_ig = 0
    maximum_threshold = 0

    for threshold in X:
        ig = information_gain(X, y, threshold)
        if ig > maximum_ig:
            maximum_ig = ig
            maximum_threshold = threshold

    return "The maximum IG = %.3f and it occured by splitting on %.4f." % (maximum_ig, maximum_threshold)

print(best_threshold(X[:, 2], y))

Let's see how we can do this with just sklearn!

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth=1, criterion="gini")
decision_tree.fit(X, y)

In [ ]:
show_decision_tree(decision_tree)

If we use this as our decision tree, how accurate is it?

In [ ]:
print("Accuracy = %.3f" % (decision_tree.score(X, y)))

Let's add one more level to our decision tree.

In [ ]:
decision_tree = DecisionTreeClassifier(max_depth=5, criterion="entropy")
decision_tree.fit(X, y)

In [ ]:
show_decision_tree(decision_tree)

In [ ]:
print("Accuracy = %.3f" % (decision_tree.score(X, y)))

# Exercise
1. Compare the best thresholds for different attributes.
2. Plot on the ``petal_length`` vs ``petal_width`` figure the lines corresponding to the best thresholds and that allow to classify the points
3. Calculate the accuracy for each one level decision tree, obtained for the different attributes
4. Now take the dataset, take randomly 2/3 of rows to create a training dataset, and the rest for the test
5. Redo the decision tree and calculate the accuracy for the training set and test set. Do you get the same decision tree as with using the whole dataset?
6. Starting from the function `best_threshold`, create a function that returns the IG for different thredholds, not simply the best one. Plot IG vs the threshold. This is always a good practice to understand how good your best threshold is.

# Generalization

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets for both X and y
X_train, X_test, y_train, y_test = train_test_split(X, y)

Now that we have split our data, let's check how well a model does when it is fit on a "training" set and then used to predict on both the training set as well as our hold out set. Remember, the model has never seen this hold out "test" set before!

In [ ]:
# Model
model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train, y_train)

print("Accuracy on training = %.3f" % model.score(X_train, y_train))
print("Accuracy on test = %.3f" % model.score(X_test, y_test))

In [ ]:
accuracies_train = []
accuracies_test = []
depths = range(1, 9)

for md in depths:
    model = DecisionTreeClassifier(max_depth=md)
    model.fit(X_train, y_train)
    
    accuracies_train.append(model.score(X_train, y_train))
    accuracies_test.append(model.score(X_test, y_test))

plt.plot(depths, accuracies_train, label="Train")
plt.plot(depths, accuracies_test, label="Test")
plt.title("Performance on train and test data")
plt.xlabel("Max depth")
plt.ylabel("Accuracy")
plt.ylim([0.85, 1.05])
plt.xlim([1,9])
plt.legend()
plt.show()

Iris actually works very well, but let's see what happens when we use some artificial data.

More details on model evaluation in the next class!

# Advanced (optional)


## Tips on pratical use

* Decision trees tend to overfit on data with a large number of features. Getting the right ratio of samples to number of features is important, since a tree with few samples in high dimensional space is very likely to overfit.

* Consider performing dimensionality reduction (PCA, ICA, or Feature selection) beforehand to give your tree a better chance of finding features that are discriminative.

* Visualise your tree as you are training by using the export function. Use *max\_depth=3* as an initial tree depth to get a feel for how the tree is fitting to your data, and then increase the depth.

* Remember that the number of samples required to populate the tree doubles for each additional level the tree grows to. Use max_depth to control the size of the tree to prevent overfitting.

* Use min_samples_split or min_samples_leaf to control the number of samples at a leaf node. A very small number will usually mean the tree will overfit, whereas a large number will prevent the tree from learning the data. Try min_samples_leaf=5 as an initial value. If the sample size varies greatly, a float number can be used as percentage in these two parameters. The main difference between the two is that min_samples_leaf guarantees a minimum number of samples in a leaf, while min_samples_split can create arbitrary small leaves, though min_samples_split is more common in the literature.

* Balance your dataset before training to prevent the tree from being biased toward the classes that are dominant. Class balancing can be done by sampling an equal number of samples from each class, or preferably by normalizing the sum of the sample weights (sample_weight) for each class to the same value. Also note that weight-based pre-pruning criteria, such as min_weight_fraction_leaf, will then be less biased toward dominant classes than criteria that are not aware of the sample weights, like min_samples_leaf.

* If the samples are weighted, it will be easier to optimize the tree structure using weight-based pre-pruning criterion such as min_weight_fraction_leaf, which ensure that leaf nodes contain at least a fraction of the overall sum of the sample weights.

* All decision trees use np.float32 arrays internally. If training data is not in this format, a copy of the dataset will be made.

* If the input matrix X is very sparse, it is recommended to convert to sparse csc_matrix before calling fit and sparse csr_matrix before calling predict. Training time can be orders of magnitude faster for a sparse matrix input compared to a dense matrix when features have zero values in most of the samples.

## Plot the decision surface of a decision tree on the iris dataset

Plot the decision surface of a decision tree trained on pairs
of features of the iris dataset.

See `decision tree <tree>` for more information on the estimator.

For each pair of iris features, the decision tree learns decision
boundaries made of combinations of simple thresholding rules inferred from
the training samples.

In [ ]:
# Parameters
n_classes = 3
plot_colors = "ryb"
plot_step = 0.02

plt.rcParams['figure.dpi'] = 120


# Load data
iris = load_iris()

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
    # We only take the two corresponding features
    X = iris.data[:, pair]
    y = iris.target

    # Train
    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X, y)

    # Plot the decision boundary
    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

    plt.xlabel(iris.feature_names[pair[0]])
    plt.ylabel(iris.feature_names[pair[1]])

    # Plot the training points
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                    cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

plt.suptitle("Decision surface of a decision tree using paired features")
plt.legend(loc='lower right', borderpad=0, handletextpad=0)
plt.axis("tight")
plt.show()